<a href="https://colab.research.google.com/github/dlsun/pods/blob/master/6.1%20K-Nearest%20Neighbors%20for%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In 2012, Brittany Wenger, a 17-year old high school student from Florida, won first place in the Google Science Fair. She built [a cloud-based application](http://cloud4cancer.appspot.com/) that predicts whether a patient has breast cancer, based on attributes observed in a biopsy of breast tissue. Her classifier, based on a type of machine learning model called a _neural network_, was 97.4% accurate, with a 99.1% sensitivity to malignancy. You will learn what all of these terms mean in this chapter.

![](https://7aba0e11-a-6b960d83-s-sites.googlegroups.com/a/googlesciencefair.com/science-fair-2012-project-64a91af142a459cfb486ed5cb05f803b2eb41354-1333130785-87/the-question/fna-benign1.png?attachauth=ANoY7cplerSFMhG2lY-0UZu7_xj8aa9pK2GfmAaIfqMRR71uCpyCArRiaoQM0HD2xyK49q2fxGRnWO3rXOXSI3IZNigWU3Yz9rOyGOttuWYTyB-0lP8seBT1vHLwZlEz1CHrYNrP_MMWYej262SgMSHA7Zycq13KcEIL1BQjKrsCukhYM5Qr-77ZaoZl1LEFRePEjHDz6YH1-XG9IGfA2XGt9EZhw4nW7Kv8ArO6X5PIU1Yngv2CCbNkkB-R9P1ryl2UE6SokdyeCXIIwcx7VaTcMn9_WvGOnFYPRJNgz1Fv_MaCsMi2k9GFj6eAHIKr0YNLZEbe_YXFfh5XaLFwQ6kS6lN_AwvNXA%3D%3D&attredirects=0) ![](https://7aba0e11-a-6b960d83-s-sites.googlegroups.com/a/googlesciencefair.com/science-fair-2012-project-64a91af142a459cfb486ed5cb05f803b2eb41354-1333130785-87/the-question/fna-malignant1.png?attachauth=ANoY7cogzpoYrWJola-0cgKr0L93qNVRq_HJQoI8z6hNksYF2h0hw3fMlx2mVKma9DEtDO3JFJWrxA8AC7ZJKT0O72UCtHvlJ6sW_Y-sFQFGiglg1tXulHeVfze8HeMmEOlVcGfuD19HeadVPLZlJoi51H-TOIUTd9EpeqPj3lh_SBZ6q7Wsv9Tp2jm8I--hyN9VRV82k-qd-3xpz1KWzVDgEzcx6EkxK1ReMFOifeHYXsP4r_e_g1UEt435460e6kz34mV9QkvXgeqIGmBIvM9SX4uY0Q5WN0RBfGN41wJTw7TqzfO5qxACQw5R6vkeDqp8MquxqDy8rgxr79C8CHXqW0YbefRxwA%3D%3D&attredirects=0)

First, we read in the data that Brittany Wenger trained her classifier on.



In [0]:
import pandas as pd

data_dir = "http://dlsun.github.io/pods/data/"
df_breast = pd.read_csv(data_dir + "breast-cancer.csv")
df_breast.head()

# 6.1 K-Nearest Neighbors for Classification

In the previous chapter, we trained machine learning models to predict quantitative labels. Such models are called _regression models_ (or _regressors_). In this chapter, we will train machine learning models to predict categorical variables. In other words, the goal is to "classify" observations as belonging to one class or the other. These models are called _classification models_ (or _classifiers_).

We focus on just two features for now: **Clump Thickness** and **Uniformity of Cell Size**. Let's plot the training data, using color to represent the class label.

In [0]:
colors = df_breast["Class"].map({
    0: "blue",
    1: "red"
})

df_breast.plot.scatter(
    x="Clump Thickness", y="Uniformity of Cell Size", c=colors, 
    alpha=.2
);

Now, suppose that we have a new sample of breast mass with clump thickness 6 and uniformity of cell size 7, represented by a black circle on the plot below. Is this likely a malignant or a benign tumor?

![](classification.png)

It is not hard to guess that this tumor is probably malignant, just by looking at the plot. The reasoning goes like this: most of the breast masses in the training data that were "close" to this one were malignant, so it makes sense to predict that this mass is also malignant. This is precisely the idea behind the $k$-nearest neighbors classifier:

1. Calculate the distance between the new point and each point in the training data, using some distance metric on the features.
2. Determine the $k$ closest points. Of these $k$ closest points, count up how many of each class label there are.
3. The predicted class of the new point is whichever class was most common among the $k$ closest points.

The only difference between the $k$-nearest neighbors classifier and the $k$-nearest neighbors regressor from the previous chapter is the last step. Instead of averaging the labels of the $k$ neighbors to obtain our prediction, we count up the number of occurrences of each category among the labels and take the most common one. It makes sense that we have to do something different because the label is now categorical instead of quantitative. This is yet another example of the general principle that was introduced in Chapter 1: the analysis changes depending on the variable type!

# Implementing K-Nearest Neighbors from Scratch

Let's classify the breast mass above using 10-nearest neighbors. First, we extract the training data and scale the features.

In [0]:
X_train = df_breast[["Clump Thickness", "Uniformity of Cell Size"]]
y_train = df_breast["Class"]

X_train_sc = (X_train - X_train.mean()) / X_train.std()

Next, we create a `Series` for the new breast mass, being sure to scale it in the exact same way.

In [0]:
x_new = pd.Series()
x_new["Clump Thickness"] = 6
x_new["Uniformity of Cell Size"] = 7

x_new_sc = (x_new - X_train.mean()) / X_train.std()
x_new_sc

Now, we calculate the (Euclidean) distance between this new mass and each mass in the training data, and sort the distances from smallest to largest.

In [0]:
import numpy as np
dists = np.sqrt(((X_train_sc - x_new_sc) ** 2).sum(axis=1))
dists_sorted = dists.sort_values()
dists_sorted

The first 10 entries of this sorted list will be the 10 nearest neighbors. Let's get their index.

In [0]:
inds_nearest = dists_sorted.index[:10]
inds_nearest

Now we can look up these indices in the original data.

In [0]:
df_breast.loc[inds_nearest]

As a sanity check, notice that these examples all have a clump thickness near 6 and a uniformity of cell size near 7. To make a prediction for the new mass, we need to count up how many malignant and benign masses there were among these 10-nearest neighbors.

In [0]:
df_breast.loc[inds_nearest]["Class"].value_counts()

There were more malignant tumors, by an 8 to 2 margin, so the 10-nearest neighbors model predicts that the breast is malignant.

As a measure of confidence in a prediction, classification models often report the predicted _probability_ of each label, instead of just the predicted label. The predicted probability of a class in a $k$-nearest neighbors model is simply the proportion of the $k$ neighbors that are in that class. In the example above, instead of simply predicting that the breast mass is malignant, we could have instead said that it has a $8/10 = .8$ probability of being malignant.

# K-Nearest Neighbors Classifier in Scikit-Learn

Now let's implement the same $10$-nearest neighbors model above using Scikit-Learn.

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline

# define a pipeline
pipeline = make_pipeline(
    StandardScaler(),
    KNeighborsClassifier(n_neighbors=10)
)

# fit the pipeline (using X_train and y_train defined above)
pipeline.fit(X_train, y_train)

# define the test data (recall that scikit-learn expects a 2D-array)
pipeline.predict([[6, 7]])

What if we want the predicted probabilities? For classification models, there is an additional method, `.predict_proba()`, that returns the predicted probability of each class.

In [0]:
pipeline.predict_proba([[6, 7]])

The first number represents the probability of the first class (0 for benign) and the second number represents the probability of the second class (1 for malignant). Notice that the predicted probabilities add up to 1, as they must.

# Exercises

1\. The wines data set (https://dlsun.github.io/pods/data/wines.csv ) contains data about the chemical composition of various wines. We want to predict the color (red or white) of a wine with the following features.

- fixed acidity: 11
- volatile acidity: 0.3
- citric acid: 0.3
- residual sugar: 2
- chlorides: 0.08
- free sulfur dioxide: 17
- total sulfur dioxide: 60
- density: 1.0
- pH: 3.2
- sulphates: 0.6
- alcohol: 9.8
- quality: 6

Train a 15-nearest neighbors model to predict the color of this wine from just it fixed acidity and volatile acidity. Then, train another 15-nearest neighbors model using all of the features. Does the predicted label change? Do the predicted probabilities of the labels change?

2\. Build a 5-nearest neighbors model to predict whether or not a passenger on a Titanic would survive, based on their age, sex, and class. Use the Titanic data set (https://dlsun.github.io/pods/data/titanic.csv ) as your training data. Then, use your model to predict whether a 20-year old female in first-class would survive. What about a 20-year old female in third-class?